In [1]:
# core imports
import os, sys, json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# adds parent directory to python path so we can access the pre-trained models in keras_models director
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path: sys.path.append(nb_dir)

In [39]:
# ml imports
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# set_session(tf.Session(config=config))

from sklearn.preprocessing import OneHotEncoder

from keras.models import Model, Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

from pretrained_models.vgg19 import VGG19
from pretrained_models.vgg16 import VGG16
from pretrained_models.imagenet_utils import preprocess_input, decode_predictions

In [20]:
import bcolz

def get_batches(dirname, gen=image.ImageDataGenerator(), target_size=(224,224), 
                class_mode='categorical', batch_size=4, shuffle=True):
    return gen.flow_from_directory(dirname, target_size, class_mode=class_mode, batch_size=batch_size, shuffle=shuffle)

def get_data(dirname, target_size=(224,224)):
    batches = get_batches(dirname, target_size=target_size, class_mode=None, batch_size=1, shuffle=False)
    return np.concatenate([ batches.next() for i in range(batches.n) ])

def finetune(mode)

def onehot(x):
    return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

def save_array(fname, arr):
    c = bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]

In [5]:
# path = "data/"
path = "data/sample/"

train_path = path + 'train/'
val_path = path + 'valid'

In [6]:
batch_size = 4 #64

In [7]:
model = VGG19(weights='imagenet', include_top=True)

In [8]:
# model.summary()

Preprocess the data

In [9]:
# use batch_size = 1 since we're just preprocessing on the CPU
train_batches = get_batches(train_path, batch_size=1, shuffle=False)
val_batches = get_batches(val_path, batch_size=1, shuffle=False)

Found 300 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [13]:
train_data = get_data(train_path)
val_data = get_data(val_path)

# print(train_data.shape)
# print(train_data.shape)

Found 300 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [17]:
save_array(path+'train_data.bc', train_data)
save_array(path+'val_data.bc', val_data)

In [18]:
train_data = load_array(path+'train_data.bc')
val_data = load_array(path+'val_data.bc')

In [28]:
train_classes = train_batches.classes
val_classes = val_batches.classes

train_labels = onehot(train_classes)
val_labels = onehot(val_classes)

Get 1,000 ImageNet probabilities from VGG to use as the features for our linear model

In [36]:
train_features = model.predict(train_data, batch_size)
val_features = model.predict(val_data, batch_size)

# print(train_features.shape)
# print(val_features.shape)

In [37]:
save_array(path+'train_lastlayer_features.bc', train_features)
save_array(path+'val_lastlayer_features.bc', val_features)

In [38]:
train_features = load_array(path+'train_lastlayer_features.bc')
val_features = load_array(path+'val_lastlayer_features.bc')

Define our linear model

In [42]:
lm = Sequential([ Dense(2, activation='softmax', input_shape=(1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [49]:
lm.fit(train_features, train_labels, epochs=3, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 300 samples, validate on 30 samples
Epoch 1/3
300/300 [==============================] - 0s - loss: 0.2371 - acc: 0.9667 - val_loss: 0.1962 - val_acc: 0.9667
Epoch 2/3
300/300 [==============================] - 0s - loss: 0.1932 - acc: 0.9633 - val_loss: 0.1638 - val_acc: 0.9667

In [48]:
# lm.summary()

In [ ]:
gen = image.ImageDataGenerator()
train_batches = gen.flow_from_directory(path+'train', target_size=(224,224), 
                                        class_mode='categorical', shuffle=True, batch_size=batch_size)

gen = image.ImageDataGenerator()
val_batches = gen.flow_from_directory(path+'valid', target_size=(224,224), 
                                        class_mode='categorical', shuffle=True, batch_size=batch_size*2)

In [ ]:
# train_batches.class_indices
# train_batches.num_class

In [ ]:
model.layers.pop()
for layer in model.layers: layer.trainable = False
    
# recover the output from the last layer in the model and use as input to new Dense layer
last = model.layers[-1].output
x = Dense(train_batches.num_class, activation="softmax")(last)
model = Model(model.input, x)

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.summary()

In [ ]:
train_steps = train_batches.n / batch_size
val_steps = val_batches.n / batch_size

model.fit_generator(train_batches, steps_per_epoch=train_steps, epochs=3, 
                    validation_data=val_batches, validation_steps=val_steps)

In [ ]:
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
imgs, labels = next(train_batches)

plots(imgs, titles=labels)

In [ ]:
preds = model.predict(imgs) # returns probabilities for each image ([ [.999, .01], [.825, .175] ])

In [ ]:
idxs = np.argmax(preds, axis=1) # returns index of highest probability for each image ([0,1,0,1])

In [ ]:
predicted_probs = [preds[i, idxs[i]] for i in range(len(idxs))] # returns probability of predicted label ([.99, 1.0])

In [ ]:
# train_batches.class_indices => {'cats': 0, 'dogs': 1}
model_classes = list(iter(train_batches.class_indices)) # returns ['cats', 'dogs']
classes = [model_classes[idx] for idx in idxs] # returns ['cats', 'dogs', 'cats', 'dogs']

In [ ]:
for i in range(len(idxs)):
    print('{0:.4f}/{1}'.format(predicted_probs[i], classes[i]))